In [7]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import nibabel as nib # 🧠 NIfTI 파일을 다루기 위한 필수 라이브러리

# --- [설정] ---
# .nii 또는 .nii.gz 파일들이 모여있는 폴더 경로
MASK_BASE_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\seg_pseudo_masks'
SAVE_CSV_PATH = './seg_bone_ratios_from_nifti.csv'

results = []

print("🦴 3D NIfTI 마스크 픽셀 지분율 추출 시작...")

# 폴더 안의 NIfTI 파일들만 싹 다 가져오기
nifti_files = [f for f in os.listdir(MASK_BASE_DIR) if f.endswith('.nii') or f.endswith('.nii.gz')]

for nifti_file in tqdm(nifti_files, desc="Processing NIfTI Volumes"):
    # 파일명에서 환자 UID 뽑아내기 (예: '1.2.826.0.1.3680043.10001.nii' -> '1.2.826.0.1.3680043.10001')
    uid = nifti_file.replace('.nii.gz', '').replace('.nii', '')
    
    file_path = os.path.join(MASK_BASE_DIR, nifti_file)
    
    # 1. NIfTI 3D 볼륨 로드
    img = nib.load(file_path)
    mask_3d = img.get_fdata() # 일반적으로 형태는 (H, W, D) 또는 (W, H, D)
    
    # RSNA 데이터의 경우 세 번째 차원(Z축, D)이 슬라이스 개수입니다.
    num_slices = mask_3d.shape[2] 
    
    # 2. 슬라이스(Z축) 단위로 김밥 썰듯 훑으며 계산
    for z in range(num_slices):
        slice_mask = mask_3d[:, :, z] # 2D 슬라이스 한 장 떼어내기
        
        # 픽셀 개수 세기 (C1~C7)
        bone_counts = {c: np.sum(slice_mask == c) for c in range(1, 8)}
        total_bone_pixels = sum(bone_counts.values())
        
        # 비율(Ratio) 계산
        ratios = {f'C{c}_ratio': 0.0 for c in range(1, 8)}
        
        if total_bone_pixels > 0:
            for c in range(1, 8):
                ratios[f'C{c}_ratio'] = bone_counts[c] / total_bone_pixels
                
        # --- [중요] 슬라이스 번호 매칭 ---
        # 보통 배열의 인덱스(0부터 시작)에 1을 더하면 원본 DICOM의 InstanceNumber(슬라이스 번호)와 일치합니다.
        slice_idx = z + 1 
        
        # 3. 결과 기록
        row_data = {
            'StudyInstanceUID': uid,
            'slice_idx': slice_idx,
            'total_bone_pixels': total_bone_pixels
        }
        row_data.update(ratios)
        
        results.append(row_data)

# --- [CSV 저장] ---
df_ratios = pd.DataFrame(results)
df_ratios = df_ratios.sort_values(by=['StudyInstanceUID', 'slice_idx']).reset_index(drop=True)
df_ratios.to_csv(SAVE_CSV_PATH, index=False)

print(f"\n✨ 완료! NIfTI에서 추출한 뼈 지분율 데이터가 저장되었습니다: {SAVE_CSV_PATH}")

🦴 3D NIfTI 마스크 픽셀 지분율 추출 시작...


Processing NIfTI Volumes: 100%|██████████| 235/235 [02:29<00:00,  1.58it/s]



✨ 완료! NIfTI에서 추출한 뼈 지분율 데이터가 저장되었습니다: ./seg_bone_ratios_from_nifti.csv


## 최종 성능

In [13]:
import pandas as pd
import numpy as np

CLF_PREDS_PATH = './runs/rsna_clf_efficientnet/val_full_predictions.csv'
SEG_RATIOS_PATH = './seg_bone_ratios_from_nifti.csv'
FINAL_SUBMISSION_PATH = './final_patient_predictions.csv'

print("🔄 1. 데이터 불러오기...")
df_clf = pd.read_csv(CLF_PREDS_PATH)
df_seg = pd.read_csv(SEG_RATIOS_PATH)

# ✂️ [핵심 해결책] 꼬리표 잘라내기! ('_mask' 글자를 빈칸으로 바꿈)
df_seg['StudyInstanceUID'] = df_seg['StudyInstanceUID'].str.replace('_mask', '', regex=False)

# 🛡️ 데이터 타입 완벽 통일
df_clf['StudyInstanceUID'] = df_clf['StudyInstanceUID'].astype(str)
df_seg['StudyInstanceUID'] = df_seg['StudyInstanceUID'].astype(str)

df_clf['slice_idx'] = df_clf['slice_idx'].astype(int)
df_seg['slice_idx'] = df_seg['slice_idx'].astype(int)

print(f"📊 분류 데이터: {len(df_clf)}장 | 마스크 데이터: {len(df_seg)}장")

print("🔄 2. 톱니바퀴 조립(Merge) 중...")
df_merged = pd.merge(df_clf, df_seg, on=['StudyInstanceUID', 'slice_idx'], how='inner')

print(f"✅ 병합 성공! 매칭된 슬라이스: {len(df_merged)}장\n")

if len(df_merged) == 0:
    print("🚨 아직도 0장이라면 'slice_idx' 대신 'original_slice_number'로 합쳐야 합니다.")
else:
    # 3. 마법의 계산: 확률 × 비율
    for c in range(1, 8):
        df_merged[f'C{c}_fracture_score'] = df_merged['pred_prob'] * df_merged[f'C{c}_ratio']

    # 4. 환자별 최종 결론 내리기 (Max Pooling)
    print("🧠 3. 환자별 최종 점수(Max Pooling) 계산 중...")
    aggregation_dict = {f'C{c}_fracture_score': 'max' for c in range(1, 8)}
    df_patient = df_merged.groupby('StudyInstanceUID').agg(aggregation_dict).reset_index()

    df_patient = df_patient.rename(columns={
        'C1_fracture_score': 'C1', 'C2_fracture_score': 'C2',
        'C3_fracture_score': 'C3', 'C4_fracture_score': 'C4',
        'C5_fracture_score': 'C5', 'C6_fracture_score': 'C6', 'C7_fracture_score': 'C7',
    })

    # 전체 환자 골절 확률 도출 (뼈 점수 중 최고점)
    df_patient['patient_overall'] = df_patient[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].max(axis=1)
    df_patient = df_patient[['StudyInstanceUID', 'patient_overall', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']]

    # 5. 저장 및 출력
    df_patient.to_csv(FINAL_SUBMISSION_PATH, index=False)
    print(f"✨ 최종 제출 파일 저장 완료! ➔ {FINAL_SUBMISSION_PATH}\n")
    
    # 결과 미리보기 출력
    print(df_patient.head().to_markdown(index=False))

🔄 1. 데이터 불러오기...
📊 분류 데이터: 10013장 | 마스크 데이터: 77724장
🔄 2. 톱니바퀴 조립(Merge) 중...
✅ 병합 성공! 매칭된 슬라이스: 10013장

🧠 3. 환자별 최종 점수(Max Pooling) 계산 중...
✨ 최종 제출 파일 저장 완료! ➔ ./final_patient_predictions.csv

| StudyInstanceUID          |   patient_overall |       C1 |       C2 |       C3 |        C4 |         C5 |        C6 |          C7 |
|:--------------------------|------------------:|---------:|---------:|---------:|----------:|-----------:|----------:|------------:|
| 1.2.826.0.1.3680043.11401 |          0.969864 | 0.969864 | 0.825633 | 0.560107 | 0.76769   | 0.853865   | 0.831188  | 0.407861    |
| 1.2.826.0.1.3680043.1151  |          0.913589 | 0.913589 | 0.883897 | 0.139199 | 0.625354  | 0.779593   | 0.0996773 | 0.015646    |
| 1.2.826.0.1.3680043.1195  |          0.994656 | 0.994656 | 0.667213 | 0.969769 | 0.0256381 | 0.800307   | 0.164653  | 0.00287244  |
| 1.2.826.0.1.3680043.11988 |          0.886296 | 0.886296 | 0.849171 | 0.321834 | 0.0242441 | 0.19577    | 0.040992  | 0.000834302 |
| 1

## 최종 점수 계산

In [15]:
import pandas as pd
import numpy as np

# --- [1. 파일 경로 설정] ---
# 방금 사용자님이 만드신 예측 결과 파일
PRED_PATH = 'final_patient_predictions.csv' 
# RSNA 대회에서 다운받았던 원본 정답지 파일
GT_PATH = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\filtered_train.csv' 

def compute_rsna_loss(pred_df, gt_df):
    # 예측한 환자들만 정답지에서 교집합으로 뽑아오기
    df = pd.merge(pred_df, gt_df, on='StudyInstanceUID', how='inner', suffixes=('_pred', '_gt'))
    
    # 캐글 채점 기준 (가중치)
    weights = {
        'C1': 1, 'C2': 1, 'C3': 1, 'C4': 1, 'C5': 1, 'C6': 1, 'C7': 1,
        'patient_overall': 7  # 전체 골절 여부가 7배 중요!
    }
    
    total_loss = 0.0
    total_weight = 0.0
    
    # 극단적인 확률값(1.0 또는 0.0)이 들어가면 로그(log) 계산 시 에러가 나므로, 살짝 깎아줍니다 (Clip)
    eps = 1e-15
    
    # 8개의 항목(C1~C7, overall)에 대해 각각 Loss 계산
    for col, w in weights.items():
        y_true = df[col].values
        # 우리가 만든 예측값 기둥 이름은 col 그대로 (예: 'C1')
        y_pred = df[col].values 
        
        # 0과 1 사이로 안전하게 가두기
        y_pred = np.clip(y_pred, eps, 1 - eps)
        
        # Log Loss 공식
        loss = - w * (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        
        total_loss += loss.sum()
        total_weight += w * len(df)
        
    final_score = total_loss / total_weight
    return final_score

# --- [2. 채점 실행!] ---
pred_df = pd.read_csv(PRED_PATH)
gt_df = pd.read_csv(GT_PATH)

# 캐글 점수 계산
kaggle_score = compute_rsna_loss(pred_df, gt_df)

print("🥁 두구두구두구...")
print(f"🔥 나의 가상 캐글 점수 (Log Loss): {kaggle_score:.4f}")
print("(점수는 낮을수록(0에 가까울수록) 좋습니다!)")

KeyError: 'C1'